# Sudoku Solver
---

Sudoku is number puzzle with 9X9 grid with 9 3X3 subgrids. Some of the squares are filled with numbers.
The objective of the puzzle is to fill all the 81 squares with numbers from 1-9 in a way that no number gets repeated in each row, column or box. 

The objective of this work is to design an agent that solves sudoku puzzle.


In [413]:
# import necessary modules 

from SudokuSolver import SudokuSolver 
%load_ext autoreload
%autoreload 2

squares   = str
gridValDict = dict
cols = '123456789'
rows      = 'ABCDEFGHI'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [414]:
def cross(A,B) -> tuple:
    return tuple(a+b for a in A for b in B)

## Representation of puzzle
---

The puzzle is represented as follows ( as mentioned in Stuart Russell and Peter Norvig's 'Artificial Intelligence: A Modern Approach' book)

    - 81 squares [A1', 'A2', ... 'I9']
    - 3X3 subgrids are represented as boxes(9 boxes)
    - units are  rows, columns and boxes(subgrids)  
    - peers of a square represents are the squares that are present in the same row or colum or subgrid

In [415]:
squares   = cross(rows, cols)
#list of tuples - [('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'),...] 
#all 3x3 boxes in puzzle - total 9
all_boxes = [cross(rs, cs)  for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')] 
#list of tuples - [('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),..]
#all  rows, columns and boxes - total 27
all_units = [cross(rows, c) for c in cols] + [cross(r, cols) for r in rows] + all_boxes

"""
dictionary holding units for each square - {'A1': (('A1',...,'I1'), ('A1',...,'A9'), ('A1',...,'C3')),
                                            'A2': (('A2',...,'I2'), ('A1',...,'A9'), ('A1',...,'C3')),
                                            .
                                            .
                                            'I9':(('A9',...,'I9'),  ('I1',...,'I9'), ('G7',...,'I9'))}

Total - 81 units
"""

units     = {s: tuple(u for u in all_units if s in u) for s in squares}
# Dictionary holding set of peers - {'A1': {'A2', 'A3',.....,I1},.....'I9':{A9,...,I8} }
# Total 81 units

peers     = {s: set().union(*units[s]) - {s} for s in squares}

In [416]:
# Module to display the Sudoku grid

def display(values):
    "Display these values as a 2-D grid."    

    width = 1+max(len(values[s]) for s in squares)
    
    
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print ( ''.join(values[r+c].center(width)+('|' if c in '36' else '')
                    for c in cols) )
        if r in 'CF': print(line)
    print()



## Constraint Satisfaction Problem
---
Constraint Satisfaction Probelm is a mathematical problem that can be solved by finding the values of all the variables with respect to the constraints. The Constraint Satisfaction problem consists of

- **Variables** - Variables are the states. In the case of Sudoku the 81 squares from A1 to I9 are variables

- **Domains**  - Domains are the set of values that can be assigned to the variables. In Sudoku, domain is the values from 1-9 which are the possible values of each state

- **Constraints** - Constraint is the set of restrictions applied on the problem. In sudoku, The numbers are constrained in a way that no number should be repeated in each Columns, Rows and subgrid(box)


Sudoku is considered as a Constrain satisfaction problem with partial assignment  since few of the grids are already filled and the remaining squares should be filled

There are different types of constraints. 
* **Unary Constraint** - Constraint on a single variable
* **Binary Constraint** - Constraint is related with two variables
* **Global Constraint** - This contains arbitrary number of variables. Alldiff constraint is a global constraint which means all the variables are different. The contraint in sudoku problem is alldiff constraint since all the variables in each unit(row, column, box) should be different.

  


## Constraint Propagation
Constraint propagation is an inference that can be used to reduce the domain values for the variable, that will inturn reduces the domain value of another variable and so on.

In the sudoku solver module, I have implemented three constraint propagation methods and backtrackking for searching the required combination.
Simple naive backtracking alone will be able to find the solution  for the sudoku puzzles. But the drawback would be the time complexity. It will search each and every combination from scratch. That would be n*9!*(81-n)! where n is the number of digits filled already filled.
Inorder to fasten the process, The constraint propagation will be helpful to reduce the domain values for some variables by appying some strategies. I applied the following strategies and compared its results.

- **Elimination Strategy**
    - If there is a value present in one of the squares (variable), then eliminate the value from all its peer's (ie squares belong to the same row/column/box) domains.


- **Single posssibility strategy**
    - In this strategy, for a square, if there is only one possible number then assign the number in that square(remove all the other values in the variables domain).
    

- **Naked pairs rule**
    - This is a little complicated strategy that will be helpful for solving sone hard puzzles. When two squares in the same unit(two/column/box) has two domain values and the same domain values, then those two values can occur in only those two squares. So those two values can be removed from domain of all the other squares that are peers.


In [421]:
# Example sudoku solved - one easy and one hard

grid1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid2 = '..53.....8......2..7..1.5..4....53...1..7...6..32...8..6.5....9..4....3......97..'

solver = SudokuSolver(squares, all_units, peers)
constraint = 2
gridValDict = solver.gridtoValues(grid2)
consProp = solver.reduceGridVal(gridValDict, constraint)
print('-'*60)
print("\tPuzzle after contraint propagation")
print('-'*60)
display(consProp)


------------------------------------------------------------
	Puzzle after contraint propagation
------------------------------------------------------------
 1269  249    5   |  3   24689 24678 |14689 14679  1478 
  8    349   169  | 4679   5    467  | 1469   2    1347 
 2369   7    269  | 4689   1    2468 |  5    469   348  
------------------+------------------+------------------
  4    289  26789 | 1689  689    5   |  3    179   127  
 259    1    289  | 489    7     3   | 249   459    6   
 5679   59    3   |  2    469   146  | 149    8    1457 
------------------+------------------+------------------
 1237   6    1278 |  5    2348 12478 | 1248   14    9   
12579  2589   4   | 1678  268  12678 | 1268   3    1258 
 1235  2358  128  | 1468 23468   9   |  7    1456 12458 



After the constraint propagation most of the easy puzzles got solved. But for the hard problem again had to do backtracking.
# Backtracking Search
Back tracking search uses a depth first search to try all values till the bottom and then backtrack if these is any inconsistency (ie no value can be assigned to the variable to satisfy the constraint)

when selecting an unassigned variable in back tracking value ordering and variable ordering should be considered. In this problem, I used minimum remaining value heuristic and also compared its peformance with static variable ordering and random ordering. For value ordering I just used all the digits in order.
<br>
Then I did forward checking which allows to eliminate all the domains that are inconsistent with the constraint, every time we assign a value to the variable in bactracking process.

In [420]:
print("\tPuzzle after backpropagation")
print('-'*60)
solution = solver.backtrack(consProp, constraint,'static')
display(solution)

	Puzzle after backpropagation
------------------------------------------------------------
1 4 5 |3 2 7 |6 9 8 
8 3 9 |6 5 4 |1 2 7 
6 7 2 |9 1 8 |5 4 3 
------+------+------
4 9 6 |1 8 5 |3 7 2 
2 1 8 |4 7 3 |9 5 6 
7 5 3 |2 9 6 |4 8 1 
------+------+------
3 6 7 |5 4 2 |8 1 9 
9 8 4 |7 6 1 |2 3 5 
5 2 1 |8 3 9 |7 6 4 



In [427]:
#An easy example solved by the sudoku solver
display(solver.gridtoValues(grid2))

123456789 123456789     5     |    3     123456789 123456789 |123456789 123456789 123456789 
    8     123456789 123456789 |123456789 123456789 123456789 |123456789     2     123456789 
123456789     7     123456789 |123456789     1     123456789 |    5     123456789 123456789 
------------------------------+------------------------------+------------------------------
    4     123456789 123456789 |123456789 123456789     5     |    3     123456789 123456789 
123456789     1     123456789 |123456789     7     123456789 |123456789 123456789     6     
123456789 123456789     3     |    2     123456789 123456789 |123456789     8     123456789 
------------------------------+------------------------------+------------------------------
123456789     6     123456789 |    5     123456789 123456789 |123456789 123456789     9     
123456789 123456789     4     |123456789 123456789 123456789 |123456789     3     123456789 
123456789 123456789 123456789 |123456789 123456789     9     |    7   

In [428]:
display( solver.solvePuzzle(grid1, 3,'static') )

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



# Performance comparison for the solver with various combinations of methods
---


For performance comparison I used the easy, hard and hardest puzzle from Peter Norvig's website.
There are 50 easy sudokus and  95 hard and 11 hardest sudokus.
<br>
I used Elimination and single possibility strategy and compared results for easy, hard and hardest puzzles.
Then I used Elimination, single possibility strategy and naked twin rule for all the sudoku puzzles.


In [405]:
import time

In [423]:
# Module to calculate time taken to solve all the puzzles in ths given file
def timetaken(filename, constraint, var_order):
    file1 = open(filename, 'r')
    Lines = file1.readlines()
    start = time.time()
    count = 0
    for line in Lines:
        count = count+1
        solver.solvePuzzle(line, constraint,var_order)
        
        
    end = time.time()
    avgTime = (end-start)/count
    return avgTime
    

From the above results, Easy puzzles are solved very fast. And When comparing the strategies, Easy and hard puzzles got solved faster with elimination and single possibility strategy. But the hardest puzzles were solved when naked twin strategy is introduced. 

In [424]:
constraint = [2,3,4]
var_order = ['min', 'rand', 'static']
print('-'*100)
print ("{:<35} {:<25} {:<25}".format("Contraint prop", "variable ordering",  "processing time in sec"))
print('-'*100)
# nodes = 0
files = ['Easy50.txt', 'Hard.txt', 'Hardest.txt']
for file in files:
    print(file)
    print("-"*10)
    for con in constraint:
        for v_or in var_order:
            timeTaken = timetaken(file,con, v_or)
            if(con == 2):
                con_txt = 'elim & single possib'
            elif(con == 3):
                con_txt = 'elim, s_pos, naked pair'
            elif(con==4):
                con_txt = 'elim, s_pos, n_pair, hidden pair'
            print ("{:<35} {:<25} {:<25} ".format(con_txt, v_or, round(timeTaken,4)))

----------------------------------------------------------------------------------------------------
Contraint prop                      variable ordering         processing time in sec   
----------------------------------------------------------------------------------------------------
Easy50.txt
----------
elim & single possib                min                       0.006                     
elim & single possib                rand                      0.0077                    
elim & single possib                static                    0.0048                    
elim, s_pos, naked pair             min                       0.0066                    
elim, s_pos, naked pair             rand                      0.0079                    
elim, s_pos, naked pair             static                    0.0092                    
elim, s_pos, n_pair, hidden pair    min                       0.0106                    
elim, s_pos, n_pair, hidden pair    rand                      0.0

**Hardest Puzzle**<br>

Peter Norvig mentioned about Inkala's 2 hardest puzzles and the performance he acheived for that puzzle is 0.01S for both. My work is able to acheive 0.01 in one of the hardest puzzle but the other one takes 0.1 seconds, which still needs improvement.

In [425]:
in_grid1 = '85...24..72......9..4.........1.7..23.5...9...4...........8..7..17..........36.4.'
in_grid2 = '..53.....8......2..7..1.5..4....53...1..7...6..32...8..6.5....9..4....3......97..'

in_grid = [in_grid1, in_grid2]
def ptime(in_grid,constraint, var_order):
    start = time.time()
    solver.solvePuzzle(in_grid, constraint,var_order)
    end = time.time()
    return (end-start)

In [426]:
print('-'*100)
print ("{:<35} {:<25} {:<25}".format("Contraint prop", "variable ordering",  "processing time in sec"))
print('-'*100)
for grid in in_grid:
    print(grid)
    for con in constraint:
        for v_or in var_order:
            timeTaken = ptime(grid, con, v_or)
            if(con == 2):
                con_txt = 'elim & single possib'
            elif(con == 3):
                con_txt = 'elim, s_pos, naked pair'
            elif(con==4):
                con_txt = 'elim, s_pos, n_pair, hidden pair'
            print ("{:<35} {:<25} {:<25}".format(con_txt, v_or, round(timeTaken,4)))


----------------------------------------------------------------------------------------------------
Contraint prop                      variable ordering         processing time in sec   
----------------------------------------------------------------------------------------------------
85...24..72......9..4.........1.7..23.5...9...4...........8..7..17..........36.4.
elim & single possib                min                       0.032                    
elim & single possib                rand                      0.0229                   
elim & single possib                static                    0.0787                   
elim, s_pos, naked pair             min                       0.1203                   
elim, s_pos, naked pair             rand                      0.04                     
elim, s_pos, naked pair             static                    0.064                    
elim, s_pos, n_pair, hidden pair    min                       0.024                    
elim, s_pos,

# Results and discussion

From the comparisons above, the following interpretations are made.
1. Elimination, single possibility with static variable ordering in back tracking is best performing in easy sudokus

2. Even though adding naked pair has the same performance, there is no significant improvement for adding another complex strategy. This may be because most of the easy problems doesnot involve complex strategies like hidden pair, naked pair, etc.

3. But for hard puzzles, 0.24 seconds is acheived when all the 4 strategies are used with static variable ordering. This is a pretty good improvement compared to 0.7 acheive by the basic strategies.

4. Hardest puzzles were able to acheive 0.0879 second without hidder pair itself.

5. While comparing Inkala's hardest puzzle with these combinations of algorithms, the best results I got were 0.01 and 0.06 both are acheived by adding naked pair strategy with the random variable ordering.

8. This clearly shows that, for hard puzzles, more complex strategies will save more time.

**Future work**
 I want to add few more complex strategies for constraint propagation and see if i can reduce the processing time any further for the hardest puzzles.


# References

1. https://en.wikipedia.org/wiki/Sudoku

2. Artificial Intelligence: A mordern approach by Stuart Russell and Peter Norvig

3. https://norvig.com/sudoku.html

4. https://www.sudokudragon.com/sudokustrategy.htm



